In [ ]:
pip install mlxtend

In [387]:
import pandas as pd
#from mlxtend.preprocessing import TransactionEncoder
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import numpy as np
import re

In [437]:
data_low = pd.read_csv('beginner_recipe.csv')
data_mid = pd.read_csv('intermediate_recipe.csv')
data = pd.concat([data_low, data_mid], ignore_index=True)

In [123]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1572 entries, 0 to 1571
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   메뉴 이름   1572 non-null   object 
 1   방법 분류   1572 non-null   object 
 2   국가 분류   1572 non-null   object 
 3   테마 분류   1572 non-null   object 
 4   난이도 분류  1572 non-null   object 
 5   칼로리     1572 non-null   float64
 6   조회수     1572 non-null   int64  
 7   분량      1572 non-null   int64  
 8   주재료 이름  1572 non-null   object 
 9   부재료 이름  1564 non-null   object 
 10  양념      1288 non-null   object 
 11  레시피     1572 non-null   object 
 12  조리시간    1572 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 159.8+ KB


In [378]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)

#pd.reset_option('display.max_columns')
#pd.reset_option('display.max_colwidth')

# 데이터 전처리

In [438]:
def safe_literal_eval(val):
    if pd.isna(val) or val == '['']':
        return []
    return ast.literal_eval(val)

def extract_ingredient_names(ingredient_list):
    result = []
    for ingredient in ingredient_list:
        if ingredient.strip():  # 요소가 비어있지 않은지 확인
            result.append(ingredient.split()[0])
    return result

def combine_lists(row):
    combined = []
    for col in ['main_ingredients', 'sub_ingredients', 'seasoning']:
        combined.extend(row[col])
    return combined


In [431]:
# 문자열을 리스트로 변환하는 함수 (NaN 값 처리 포함)
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return []
    
# 문자열에서 []로 감싸진 부분을 제거하는 함수
def remove_bracketed_parts(ingredient_str):
    # 정규 표현식을 사용하여 []로 감싸진 부분을 제거
    cleaned_str = re.sub(r'\[.*?\]', '', ingredient_str)
    return cleaned_str

### 모든 걸 분리하는 경우

# # 문자열을 여러 구분자로 분할하는 함수
# def split_ingredients(ingredient_str):
#     # 여러 구분자를 사용하여 문자열을 분할
#     parts = re.split(r'[ ,\(\)]', ingredient_str)
#     # 빈 문자열 및 숫자/단위 정보 제거
#     parts = [part for part in parts if part and not re.match(r'^\d', part)]
#     return parts

# # 재료명을 추출하는 함수
# def extract_ingredient_names(ingredient_list):
#     result = []
#     for item in ingredient_list:
#         # []로 감싸진 부분 제거
#         cleaned_item = remove_bracketed_parts(item)
#         # 괄호와 공백으로 분리된 각 부분을 처리
#         ingredients = split_ingredients(cleaned_item)
#         result.extend(ingredients)
#     return result


# # 문자열을 여러 구분자로 분할하는 함수
# def split_ingredients(ingredient_str):
#     # 여러 구분자를 사용하여 문자열을 분할, 괄호 안의 내용을 포함하여 분할
#     parts = re.split(r'[\s,]+', ingredient_str)
#     # 빈 문자열 제거
#     parts = [part for part in parts if part]
#     return parts

# 문자열을 여러 구분자로 분할하는 함수
def split_ingredients(ingredient_str):
    # 괄호 안의 내용을 포함하여 분할
    parts = re.findall(r'[^,\s]+(?:\([^)]*\))?', ingredient_str)
    return parts

# 재료명을 추출하는 함수
def extract_ingredient_names(ingredient_list):
    result = []
    for item in ingredient_list:
        # []로 감싸진 부분 제거
        cleaned_item = remove_bracketed_parts(item)
        # 괄호와 공백으로 분리된 각 부분을 처리
        ingredients = split_ingredients(cleaned_item)
        result.extend(ingredients)
    return result

# def remove_numeric_ingredients(ingredient_list):
#     return [ingredient for ingredient in ingredient_list if not re.match(r'^\d', ingredient)]

def remove_numeric_ingredients(ingredient_list):
    return [ingredient for ingredient in ingredient_list if not re.match(r'^[\d½¼¾⅓⅔⅛⅜⅝⅞]', ingredient)]


def extract_ingredient_names(ingredient_str):
    # 문자열을 리스트로 변환
    ingredient_list = re.findall(r"'([^']*)'", ingredient_str)
    result = []
    for item in ingredient_list:
        # 괄호 안의 내용을 포함한 문자열 추출
        item = re.sub(r'\s*\d+[^)]*', '', item)  # 숫자와 단위를 제거
        result.append(item.strip())
    return result


In [433]:
extract_unique_ingredients(data['주재료 이름'].apply(extract_ingredient_names))

{'닭(다리살)',
 '청양고추',
 '잣가루',
 '다진청고추',
 '설탕ⓑ',
 '물(미온수)',
 '쑥갓',
 '식용유 약간',
 '새우',
 '파인애플',
 '앤다이브',
 '사탕(색깔별로)',
 '브랜디',
 '샐러드오일 약간',
 '강력분',
 '비트',
 '파마산가루',
 '스팸',
 '오징어먹물',
 '적양파',
 '다시마()',
 '시판용아이스크림',
 '생우동',
 '갓',
 '게살소스',
 '유부',
 '다진건포도',
 '스타키스트참치',
 '통아몬드',
 '코코아',
 '치즈(연질)',
 '보리',
 '케일',
 '돼지고기(목심)',
 '캄파리',
 '사과',
 '쇠고기(간것)',
 '비타민',
 '타피오카전분',
 '레몬',
 '영양부추 적당량',
 '돼지고기(로스용)',
 '춘장',
 '돼지고기(등심 슬라이스)',
 '갑오징어',
 '돼지고기(사태)',
 '과일통조림 약간',
 '햄버거빵',
 '고기(산적용)',
 '고추(삭힌것)',
 '쇠고기(스테이크용안심)',
 '아몬드(갈은것)',
 '당근',
 '노란파프리카',
 '개고기',
 '참치(캔)',
 '오보로 약간',
 '껍질콩',
 '검은깨 약간',
 '바케트빵',
 '계핏가루',
 '베이킹파우더',
 '푸질리(파스타)',
 '대나무잎',
 '하이포크팜햄',
 '기타 제철 과일',
 '찐단호박',
 '땅콩버터',
 '아스파라거스',
 '패주',
 '로메인',
 '삼겹살(삼겹살)',
 '콩',
 '샐러드유',
 '건포도',
 '미나리 몇줄기',
 '사과주스',
 '당근(잘게 썬것)',
 '닭안심(안심)',
 '닭()',
 '오렌지주스',
 '오징어',
 '설탕에절인과일(설탕에절인것)',
 '라면',
 '생대구',
 '치즈(피자용)',
 '밀가루반죽(레시피참고)',
 '쌈장',
 '미역',
 '머위',
 '찹쌀()',
 '초콜릿 약간',
 '다진돼지고기(우둔)',
 '후춧가루 약간',
 '당면(불린것)',
 '날콩가루',
 '쪽파',
 '오레가노',
 '중합',
 '두부 반모',
 '총

In [439]:
data['main_ingredients'] = data['주재료 이름'].apply(safe_literal_eval) #list 화
data['main_ingredients'] = data['main_ingredients'].apply(extract_ingredient_names) #요소별 재료 추출
data['main_ingredients'] = data['main_ingredients'].apply(remove_numeric_ingredients)

data['sub_ingredients'] = data['부재료 이름'].apply(safe_literal_eval)
data['sub_ingredients'] = data['sub_ingredients'].apply(extract_ingredient_names)
data['sub_ingredients'] = data['sub_ingredients'].apply(remove_numeric_ingredients)


data['seasoning'] = data['양념'].apply(safe_literal_eval)
data['seasoning'] = data['seasoning'].apply(extract_ingredient_names)
data['seasoning'] = data['seasoning'].apply(remove_numeric_ingredients)


# 새로운 칼럼에 합쳐진 리스트 저장
data['whole_ingredients'] = data.apply(combine_lists, axis=1)
data['whole_ingredients']

0                                                    [백미, 은행, 밤, 대추, 잣, 물]
1                           [굴, 쌀, 보리쌀, 어묵(찐것), 표고버섯, 당근, 완두콩(통조림), 물, 소금]
2                              [김치(잘게썬것), 밥, 감자, 양파, 햄, 식용유, 계란, 소금, 후춧가루]
3                   [상추, 밥, 다진쇠고기, 다진양파, 간장, 설탕, 다진파, 다진마늘, 깨소금, 참기름, 고추장]
4                                                       [녹두, 쌀, 물, 소금, 설탕]
                                       ...                                
1567    [감자(큰것), 치즈(피자용), 찬밥, 하이포크팜햄, 방울토마토, 피망, 식용유, 양파, 토마토케찹, 소금, 후춧가루]
1568                              [가래떡(흰색), 베이컨, 옥수수, 피망, 양파, 피자치즈, 토마토케첩]
1569                              [감자(큰것), 베이컨, 방울토마토, 프레시파머산치즈, 소금, 후춧가루]
1570                       [쌀국수, 바지락, 파프리카, 숙주, 식용유, 다진마늘, 말린고추, 해선장, 굴소스]
1571                      [쇠고기, 양송이, 피망, 양파, 당근, 케첩, 우스터소스, 설탕, 미림, 파슬리가루]
Name: whole_ingredients, Length: 1572, dtype: object

In [435]:
def extract_unique_ingredients(ingredient_lists):
    unique_ingredients = set()
    for ingredients in ingredient_lists:
        unique_ingredients.update(ingredients)
    return unique_ingredients

# 전체 데이터프레임에서 중복 없는 재료 추출
extract_unique_ingredients(data['whole_ingredients'])
#unique_ingredients = 

{'잣가루',
 '다진청고추',
 '설탕ⓑ',
 '장어소스',
 '물(미온수)',
 '소다수(차가운것)',
 '새우',
 '청량고추',
 '양유치즈가루',
 '브랜디',
 '마른홍고추',
 '파마산가루',
 '다시간장',
 '적양파',
 '장식용견과류',
 '상추(쌈용)',
 '바닐라빈',
 '코코아',
 '수삼잔뿌리',
 '돼지고기(목심)',
 '겨자소스',
 '춘장',
 '타라곤',
 '초콜릿(장식용)',
 '피쉬소스(간장)',
 '땅콩버터',
 '썬것)',
 '백년초가루',
 '사과주스',
 '양파(다진',
 '넛트맥',
 '계란지단',
 '발사믹',
 '케찹',
 '레몬껍질(껍질간것)',
 '대구뼈',
 '설탕(노른자)',
 '돼지고기(살고기)',
 '코코아(가루)',
 '소가소스',
 '당면(불린것)',
 '모양틀',
 '후르츠칵테일(통조림)',
 '크림소스',
 '호두',
 '그라나',
 '럼주',
 '슬라이스피클',
 '레드커리페이스트',
 '돼기고기(안심)',
 '제철',
 '콩국(메주콩100g',
 '옥수수(통조림)',
 '황파프리카',
 '쇠고기(우둔)',
 '초콜릿(다진것)',
 '소금(도우)',
 '파슬리가루',
 '바닐라오일',
 '돼지고기',
 '초밥',
 '얼음물',
 '양지머리',
 '유부(시판용)',
 '들깨',
 '팥앙금',
 '중국부추',
 '쇠고기(사태)',
 '정도',
 '샐러드잎(푸른색)',
 '멸치젓',
 '소보로',
 '쇠고기육수',
 '각얼음',
 '꼴뚜기',
 '메주콩',
 '녹말물',
 '키조개',
 '팥',
 '덧가루(밀가루)',
 '대파(10cm)',
 '넛맥(간것)',
 '흑임자',
 '돼지고기(목살)',
 '근대',
 '육수',
 '돼기고기',
 '가쓰오브시',
 '백후추',
 '식용색소',
 '당근(5cm)',
 '샤프란',
 '바닐라에센스',
 '스파게티면',
 '재첩',
 '고춧가루(시치미)',
 '다진쇠고기',
 '피넛(다진',
 '참깨',
 '데리야끼소스',
 '다진샐러리',
 '새우젓',
 '돼

In [405]:
# 특정 요소를 리스트에서 제거하는 함수
def remove_specific_ingredient(ingredient_list, ingredient_to_remove):
    return [ingredient for ingredient in ingredient_list if ingredient != ingredient_to_remove]

# 데이터 처리: 특정 요소를 제거
ingredient_to_remove = '간'
data['whole_ingredients_remove'] = data['whole_ingredients'].apply(lambda x: remove_specific_ingredient(x, ingredient_to_remove))


In [430]:
specific_ingredient = '샐러드(치커리'

data[data['whole_ingredients'].apply(lambda ingredients: specific_ingredient in ingredients)][['메뉴 이름', '레시피', 'whole_ingredients', '주재료 이름', '부재료 이름', '양념']]
# data[data['whole_ingredients'].apply(lambda ingredients: specific_ingredient in ingredients)]['whole_ingredients']


,메뉴 이름,레시피,whole_ingredients,주재료 이름,부재료 이름,양념
1238,파프리카향의쁘띠첼샐러드,"['1. 쁘띠첼은 적당한 크기로 썰어놓는다.', '2. 치커리, 적채, 허브 등의 채소를 손으로 적당하게 찢어 찬물에 담가둔다.', '3. 주황색 파프리카를 1/3정도 잘라 내어 속을 파낸다.', '4. 썰어놓은 쁘띠첼를 ③번 파프리카 안에 채워 넣는다.', '5. 감자를 돌려깎기 해서 얇게 채썰어 놓는다.', '6. 당면은 길이 5cm 정도로 잘라서 감자와 함께 기름에 바삭하게 튀긴다.', '7. ④번 쁘띠첼에 채워 놓은 파프리카 위에 튀긴 감자와 당면을 보기좋게 올려 장식한다.', '8. 접시에 파프리카를 놓고 그 주위에 채소, 방울토마토로 장식한다.', '9. 마요네즈, 양겨자, 설탕, 파프리카 파우더로 소스를 만들어 접시에 뿌린 후 케이퍼로 장식한다.']","[쁘띠첼, 파프리카(주황색), 샐러드(치커리, 적채, 허브, 등), 방울토마토, 케이퍼, 감자, 마요네즈, 양겨자, 파프리카파우더, 설탕, 약간]","['쁘띠첼 2개', '파프리카(주황색) 1개', '샐러드(치커리', '적채', '허브 등) 50g']","['방울토마토 2개', '케이퍼 4알', '감자 1/2개']","['마요네즈 1큰술', '양겨자 1/2큰술', '파프리카파우더 1/2작은술', '설탕 약간']"


In [ ]:
# 맵기 정도를 나타내는 단어들
spicy_keywords = ['매운', '고추', '청양고추', '고춧', '김치']

# 맵기 정도 추출 함수
def extract_spiciness(description, spicy_keywords):
    for word in spicy_keywords:
        if word in description:
            return '매운'
    return '안 매운'

# 맵기 정도를 새로운 칼럼에 저장
df['spiciness'] = df['description'].apply(extract_spiciness, spicy_keywords=spicy_keywords)


In [ ]:
def calculate_spiciness(row):
    score = 0
    spicy_ingredients = ['김치', '고추', '고춧']
    
    # 주재료 이름에서 맵기 성분 찾기
    if isinstance(row['주재료 이름'], str):
        row['주재료 이름'] = eval(row['주재료 이름'])
    
    if isinstance(row['주재료 이름'], list):
        for ingredient in row['주재료 이름']:
            if any(spicy in ingredient for spicy in spicy_ingredients):
                score += 3
                
    # 부재료 이름에서 맵기 성분 찾기
    if isinstance(row['부재료 이름'], str):
        row['부재료 이름'] = eval(row['부재료 이름'])
        
    if isinstance(row['부재료 이름'], list):
        for ingredient in row['부재료 이름']:
            if any(spicy in ingredient for spicy in spicy_ingredients):
                score += 2
    
    # 양념에서 맵기 성분 찾기
    if isinstance(row['양념'], str):
        row['양념'] = eval(row['양념'])
    
    if isinstance(row['양념'], list):
        for ingredient in row['양념']:
            if any(spicy in ingredient for spicy in spicy_ingredients):
                score += 1
                
    return score

data['맵기'] = data.apply(calculate_spiciness, axis=1)
data

# 사용자 정보 반영 필터링

기본 필터링: 알레르기 재료 필터링

In [353]:
allergic_ingredients = ['계란', '달걀']
max_cooking_time = 30 #조리시간을 여기서 필터링해도 될듯

In [335]:
def exclude_allergic_ingredients(row, allergic_ingredients):
    return not any(ingredient in row['whole_ingredients'] for ingredient in allergic_ingredients)

In [344]:
recommendation_df = data[data.apply(exclude_allergic_ingredients, axis=1, allergic_ingredients=allergic_ingredients)]

## 추천시스템 A 필터링

방법(1) user_ingredients가 whole_ingredients를 모두 포함하는 경우만 filtering

In [441]:
user_ingredients = ['굴', '쌀', '대추', '보리쌀', '콩', '생강', '마늘', '현미', '잣', '호두', '아몬드', '고추', '참깨', '된장', '쌀국수', '바지락', '파프리카', '숙주', '식용유', '다진마늘', '말린고추', '해선장', '굴소스']
user_ingredients = ['달걀', '우유', '방울토마토', '케찹', '아이스크림', '소금', '콜라', '라면']

In [264]:
#recommendation_df[recommendation_df['whole_ingredients'].apply(lambda x: all(ingredient in x for ingredient in user_ingredients))].head()

In [442]:
def contains_all_ingredients(row, user_ingredients):
    return set(row['whole_ingredients']).issubset(set(user_ingredients))

recommendation_A_list_1 = recommendation_df[recommendation_df.apply(contains_all_ingredients, axis=1, user_ingredients=user_ingredients)]
recommendation_A_list_1[['메뉴 이름', 'whole_ingredients']]

,메뉴 이름,whole_ingredients


방법(2) 사용자 재료와 가장 많이 겹치는 레시피 순으로 추출

In [443]:
def count_matching_ingredients(row, user_ingredients):
    recipe_ingredients = set(row['whole_ingredients'])
    matching_ingredients_count = len(recipe_ingredients & set(user_ingredients))
    return matching_ingredients_count

# 각 레시피와 사용자가 가진 재료의 겹치는 수를 계산
recommendation_df['matching_count'] = recommendation_df.apply(count_matching_ingredients, axis=1, user_ingredients=user_ingredients)

# 겹치는 수가 많은 순서로 정렬
df_sorted = recommendation_df.sort_values(by='matching_count', ascending=False)

# 겹치는 수가 0보다 큰 레시피만 필터링
df_filtered = df_sorted[df_sorted['matching_count'] > 0]

#Top20 filtering
recommendation_A_list_2 = df_filtered[0:20]
recommendation_A_list_2[['메뉴 이름', 'whole_ingredients', 'matching_count']]

,메뉴 이름,whole_ingredients,matching_count
712,시금치크림소스푸질리,"[푸질리(파스타), 시금치, 바지락, 양송이, 방울토마토, 올리브오일, 양파, 우유, 버터, 밀가루, 육두구, 소금, 후춧가루, 베사멜라]",3
519,레이즌초코쿠키,"[밀가루(박력분), 다크초콜릿(커버춰), 버터, 무가당카카오가루, 설탕, 베이킹파우더, 소금, 우유, 건포도, 청크초콜릿(장식용)]",2
1248,스카치에그,"[쇠고기(늑간살), 돼지고기, 빵가루, 우유, 당근, 삶은, 밀가루, 계란(풀어놓은것), 빵가루, 튀김기름, 시금치, 소금, 후춧가루, 우스터소스, 설탕, 케첩]",2
1182,단호박크림수프,"[단호박, 양파, 버터, 밀가루, 물, 우유, 생크림, 소금, 설탕, 흰후추]",2
701,브루스케타,"[블루치즈(고르곤졸라), 크림치즈, 바게트(슬라이스), 생크림, 견과류(호두, 아몬드), 샐러리(다진것), 방울토마토, 바질잎, 흰후춧가루, 다진마늘, 올리브오일(엑스트라버진), 레몬즙, 설탕, 소금]",2
1214,햄감자그라탕,"[햄, 감자, 옥수수(통조림), 양송이버섯, 피자치즈, 버터, 우유, 소금]",2
1185,옥수수수프,"[옥수수(알갱이), 양파, 우유, 버터, 소금]",2
572,버섯리조또,"[쌀(불린쌀), 물, 우유, 파마산치즈, 베이컨, 인스턴트크림수프, 양송이버섯, 마늘, 화이트와인, 소금, 후춧가루, 바질]",2
1085,뚝배기짬뽕라면,"[라면, 홍합, 오징어, 양파, 호박, 당근, 다진마늘, 소금, 국간장, 참기름]",2
571,레이즌파르미자나,"[가지, 감자, 모짜렐라치즈, 파슬리, [건포도라구소스], 다진건포도, 다진쇠고기, 레드와인, 토마토소스, 토마토페이스트, 다진마늘, 육수, 오레가노, 소금, 후추, [베사멜소스], 버터, 우유, 생크림, 넛맥, 소금, 후춧가루]",2


## 추천시스템 B 필터링

In [349]:
#앞과 동일
#user_ingredients = ['굴', '쌀', '대추', '보리쌀', '콩', '생강', '마늘', '현미', '잣', '호두', '아몬드', '고추', '참깨', '된장', '쌀국수', '바지락', '파프리카', '숙주', '식용유', '다진마늘', '말린고추', '해선장', '굴소스']
market_ingredients = ['보리쌀', '콩', '생강', '마늘', '현미', '잣', '호두', '아몬드', '고추', '참깨', '된장', '오징어']

In [350]:
def filter_recipes(row, user_ingredients, market_ingredients, max_additional_items=5):
    recipe_ingredients = set(row['whole_ingredients'])
    
    user_ingredients_set = set(user_ingredients)
    market_ingredients_set = set(market_ingredients)
    
    # 레시피에서 부족한 재료들 (마켓이든 어디서든 추가로 구매해야함)
    missing_ingredients = recipe_ingredients - user_ingredients_set
    
    # 부족한 재료 중 마켓에서 구매 가능한 재료
    purchasable_items_in_market = missing_ingredients & market_ingredients_set

    # 부족한 재료 중 마켓에서 구하지 못하는 재료 (purchasable_ingredients_in_market의 여집합)
    additional_items_needed = missing_ingredients - market_ingredients_set

    # 마켓이든 어디든 추가로 구매해야 하는 재료의 수가 max_additional_items 이하인 경우만 True
    can_make = len(missing_ingredients) <= max_additional_items

    return can_make, missing_ingredients, purchasable_items_in_market, additional_items_needed


# 필터링된 데이터프레임 생성
recommendation_df['can_make'], recommendation_df['missing_ingredients'], recommendation_df['purchasable_items_in_market'], recommendation_df['additional_items_needed']= zip(*recommendation_df.apply(filter_recipes, axis=1, user_ingredients=user_ingredients, market_ingredients=market_ingredients, max_additional_items=5))
recommendation_B_list = recommendation_df[recommendation_df['can_make']==True]

# 그 중 사용자가 가지고 있는 재료와 가장 겹치는게 많은 레시피 Top 20
recommendation_B_list = recommendation_B_list.sort_values(by='matching_count', ascending=False)[0:20]



/var/folders/rb/31f78rzs3r79zqkvbbxk32bw0000gn/T/ipykernel_13551/1072277277.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommendation_df['can_make'], recommendation_df['missing_ingredients'], recommendation_df['purchasable_items_in_market'], recommendation_df['additional_items_needed']= zip(*recommendation_df.apply(filter_recipes, axis=1, user_ingredients=user_ingredients, market_ingredients=market_ingredients, max_additional_items=5))


In [351]:
recommendation_B_list[['메뉴 이름', 'whole_ingredients', 'matching_count', 'missing_ingredients', 'purchasable_items_in_market', 'additional_items_needed']]

,메뉴 이름,whole_ingredients,matching_count,missing_ingredients,purchasable_items_in_market,additional_items_needed
1570,볶음쌀국수,"[쌀국수, 바지락, 파프리카, 숙주, 식용유, 다진마늘, 말린고추, 해선장, 굴소스]",9,{},{},{}
885,햄마늘볶음,"[햄, 마늘, 양송이버섯, 표고버섯, 청피망, 파프리카, 식용유, 굴소스, 참기름]",4,"{청피망, 표고버섯, 양송이버섯, 참기름, 햄}",{},"{청피망, 표고버섯, 양송이버섯, 참기름, 햄}"
108,고등어된장구이,"[고등어, 된장, 생강, 마늘, 정종, 양파, 후춧가루]",3,"{양파, 후춧가루, 고등어, 정종}",{},"{양파, 후춧가루, 고등어, 정종}"
895,대나무쌈밥,"[닭, 찹쌀, 쌀, 대나무잎, 밤, 대추, 굴소스, 소금]",3,"{소금, 밤, 닭, 대나무잎, 찹쌀}",{},"{소금, 밤, 닭, 대나무잎, 찹쌀}"
665,고춧기름,"[식용유, 마른고추, 생강, 고춧가루(굵은것), 다진마늘, 파]",3,"{고춧가루(굵은것), 마른고추, 파}",{},"{고춧가루(굵은것), 파, 마른고추}"
272,배숙,"[배, 잣, 통후추, 생강, 흑설탕, 흰설탕, 물]",2,"{배, 통후추, 물, 흰설탕, 흑설탕}",{},"{배, 물, 흑설탕, 흰설탕, 통후추}"
156,소꼬리찜,"[소꼬리, 사골, 대추, 밤, 마늘, 청양고추, 대파]",2,"{청양고추, 대파, 사골, 밤, 소꼬리}",{},"{청양고추, 대파, 사골, 밤, 소꼬리}"
162,티포크쌈,"[돼지고기, 된장, 김치, 녹차, 마늘, 새우젓]",2,"{새우젓, 김치, 녹차, 돼지고기}",{},"{새우젓, 돼지고기, 녹차, 김치}"
190,숙주미나리무침,"[숙주, 미나리, 홍고추, 다진파, 다진마늘, 깨소금, 참기름]",2,"{다진파, 깨소금, 미나리, 홍고추, 참기름}",{},"{다진파, 깨소금, 미나리, 홍고추, 참기름}"
222,마늘쫑굴소스볶음,"[마늘쫑, 새우(마른것), 식용유, 간장, 굴소스, 참기름]",2,"{참기름, 새우(마른것), 간장, 마늘쫑}",{},"{참기름, 새우(마른것), 간장, 마늘쫑}"


# 코사인 유사도 계산

In [ ]:
allergic_ingredients = ['계란', '달걀']
max_cooking_time = 30 #조리시간을 여기서 필터링해도 될듯
user_ingredients = ['굴', '쌀', '대추', '보리쌀', '콩', '생강', '마늘', '현미', '잣', '호두', '아몬드', '고추', '참깨', '된장', '쌀국수', '바지락', '파프리카', '숙주', '식용유', '다진마늘', '말린고추', '해선장', '굴소스']


In [97]:
tfidf = TfidfVectorizer()
ingredient_vectors = tfidf.fit_transform(data['main_ingredients'])
nation_vectors = tfidf.fit_transform(data['main_ingredients'])


In [98]:
scaler = StandardScaler()
numerical_data = scaler.fit_transform(data[['조회수', '칼로리']])

In [104]:
# 가중치 설정
ingredient_weight = 0.8
views_weight = 0.1
calories_weight = 0.1

ingredient_vectors_weighted = ingredient_vectors.toarray() * ingredient_weight
numerical_data_weighted = numerical_data * np.array([views_weight, calories_weight])

# TF-IDF 벡터와 수치 데이터를 결합
combined_vectors = np.hstack([ingredient_vectors_weighted, numerical_data_weighted])

# 코사인 유사도 계산
similarity_matrix = cosine_similarity(combined_vectors)
similarity_df = pd.DataFrame(similarity_matrix, index=data['메뉴 이름'], columns=data['메뉴 이름'])

In [105]:
similarity_df

메뉴 이름,견과류 영양솥밥,굴밥,김치볶음밥,꽃상추쌈,녹두죽,단팥죽,달래간장과 모둠버섯밥,닭날개주먹밥,닭안심죽,대구계란죽,...,김치계란피자,김치치킨도리아,대파 파프리카굴소스볶음,돈육불고기,두부케첩볶음,삶은무해산물볶음,엔다이브 위의 쇠고기볶음,피자떡볶이,해물채소볶음,햄마늘볶음
메뉴 이름,,,,,,,,,,,,,,,,,,,,,
견과류 영양솥밥,1.000000,0.002826,0.012735,0.004700,0.001424,-0.004278,0.010077,0.011266,0.000863,0.000619,...,0.003710,0.019115,-0.008905,-0.001138,-0.005098,-0.003484,0.006614,0.000466,-0.005635,-0.003388
굴밥,0.002826,1.000000,0.022912,0.006846,0.001930,-0.004322,0.010376,0.012477,0.001571,0.000206,...,0.003206,0.022030,-0.011576,-0.001707,-0.004258,-0.005558,0.006250,-0.000378,-0.007198,-0.004170
김치볶음밥,0.012735,0.022912,1.000000,0.027668,0.007901,-0.018804,0.044948,0.053182,0.006130,0.001320,...,0.430138,0.093108,-0.047762,-0.006864,-0.019428,-0.022125,0.027631,-0.000787,-0.029801,-0.017391
꽃상추쌈,0.004700,0.006846,0.027668,1.000000,0.002603,-0.006677,0.015879,0.018434,0.001889,0.000641,...,0.005371,0.031943,-0.015899,-0.002208,-0.007265,-0.007019,0.009989,0.000069,-0.009963,-0.005868
녹두죽,0.001424,0.001930,0.007901,0.002603,1.000000,-0.001999,0.004747,0.005477,0.000539,0.000208,...,0.001629,0.009460,-0.004662,-0.000640,-0.002209,-0.002024,0.003007,0.000053,-0.002926,-0.001728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
삶은무해산물볶음,-0.003484,-0.005558,-0.022125,-0.007019,-0.002024,0.005029,-0.011985,-0.014024,-0.001512,-0.000429,...,-0.003976,-0.024407,0.012305,0.001735,0.005358,1.000000,-0.007468,0.000056,0.007697,0.004515
엔다이브 위의 쇠고기볶음,0.006614,0.006250,0.027631,0.009989,0.003007,-0.008843,0.020857,0.023433,0.001874,0.001226,...,0.007600,0.039870,-0.018745,-0.002425,-0.010423,-0.007468,1.000000,0.000854,-0.011845,-0.007101
피자떡볶이,0.000466,-0.000378,-0.000787,0.000069,0.000053,-0.000490,0.001110,0.001045,-0.000058,0.000165,...,0.000546,0.001580,-0.000443,-0.000006,-0.000783,0.000056,0.000854,1.000000,-0.000309,-0.000221


In [106]:
def get_top_similar_recipes(similarity_df, recipe_name, top_n=5):
    # 특정 메뉴에 대한 유사도 값 추출 및 정렬
    similar_scores = similarity_df[recipe_name].sort_values(ascending=False)
    # 상위 top_n 메뉴 추출 (자기 자신 제외)
    top_similar = similar_scores.iloc[1:top_n+1]
    return top_similar

In [109]:
# 예시: 'recipe1'에 대해 가장 높은 유사도 값을 갖는 top 5 메뉴 추출
top_5_similar = get_top_similar_recipes(similarity_df, '피자떡볶이', top_n=5)
print(top_5_similar)

메뉴 이름
오징어떡볶음      0.824627
베이컨떡말이조림    0.780263
자장떡볶이       0.590539
무죽          0.009109
삼색라이스케이크    0.007149
Name: 피자떡볶이, dtype: float64


In [12]:
all_ingredients = data['방법 분류'].explode()
unique_ingredients = all_ingredients.unique()
print(unique_ingredients)

["['밥', '죽', '스프']" "['국']" "['조림']" "['구이']" "['튀김', '커틀릿']" "['찜']"
 "['나물', '생채', '샐러드']" "['밑반찬', '김치']" "['도시락', '간식']" "['양념장']"
 "['떡', '한과']" "['빵', '과자']" "['음료']" "['만두', '면류']" "['찌개', '전골', '스튜']"
 "['부침']" "['볶음']" "['양식']" "['후식']" "['그라탕', '리조또']" "['샌드위치', '햄버거']"
 "['피자']"]


In [11]:
all_ingredients = data['국가 분류'].explode()
unique_ingredients = all_ingredients.unique()
print(unique_ingredients)

['한식' '서양' '일본' '중국' '동남아시아' '이탈리아' '멕시코' '퓨전']


In [16]:
data['테마 분류'] = data['테마 분류'].apply(ast.literal_eval)
all_ingredients = data['테마 분류'].explode()



ValueError: malformed node or string: ['손님접대', '보양식', '휴일요리']

In [17]:
data['테마 분류'].head()

0               [손님접대, 보양식, 휴일요리]
1                          [휴일요리]
2    [일상식, 연인식, 어린이간식, 휴일요리, 도시락]
3                     [일상식, 아침식사]
4               [이유식, 아침식사, 다이어트]
Name: 테마 분류, dtype: object

In [13]:
all_ingredients = data['테마 분류'].explode()
unique_ingredients = all_ingredients.unique()
print(unique_ingredients)

["['손님접대', '보양식', '휴일요리']" "['휴일요리']"
 "['일상식', '연인식', '어린이간식', '휴일요리', '도시락']" "['일상식', '아침식사']"
 "['이유식', '아침식사', '다이어트']" "['손님접대', '일상식', '이유식', '아침식사', '어린이간식']"
 "['보양식', '일상식']" "['도시락']" "['일상식', '이유식', '아침식사', '다이어트']" "['이유식']"
 "['손님접대', '일상식', '어른생일상', '휴일요리', '집들이']" "['일상식']" "['채식', '향토음식']"
 "['']" "['일상식', '아침식사', '어린이간식', '도시락']" "['일상식', '어른생일상', '휴일요리']"
 "['이유식', '아침식사', '어린이간식', '다이어트']" "['손님접대', '도시락']" "['맥주술상', '소주술상']"
 "['손님접대', '일상식', '어른생일상']" "['손님접대', '휴일요리', '도시락']" "['보양식', '어른생일상']"
 "['일상식', '이유식', '아침식사']" "['손님접대', '향토음식']" "['일상식', '어린이간식', '도시락']"
 "['일상식', '향토음식']" "['향토음식', '휴일요리', '다이어트']"
 "['보양식', '아침식사', '어린이간식', '다이어트']"
 "['어린이생일상', '아침식사', '어린이간식', '휴일요리', '도시락']"
 "['일상식', '이유식', '아침식사', '어린이간식', '다이어트']"
 "['일상식', '이유식', '아침식사', '어린이간식']" "['향토음식', '도시락']"
 "['일상식', '장마식', '아침식사']" "['일상식', '휴일요리']" "['향토음식']" "['손님접대']"
 "['이유식', '아침식사']" "['손님접대', '휴일요리']" "['채식', '일상식']"
 "['일상식', '아침식사', '다이어트']" "['일상식', '소주술상']" "['손님접대', '일상식', 

In [46]:

data = pd.get_dummies(data, columns=['국가 분류'])
columns_to_drop = ['방법 분류', '테마 분류', '난이도 분류', '조회수', '분량', '레시피', '조리시간']
data = data.drop(columns=columns_to_drop)
data['칼로리'] = data['칼로리'].apply(lambda x: 1 if x < 300 else 0)
data.rename(columns={'칼로리': '다이어트'}, inplace=True)

In [47]:
def calculate_spiciness(row):
    score = 0
    spicy_ingredients = ['김치', '고추', '고춧']
    
    # 주재료 이름에서 맵기 성분 찾기
    if isinstance(row['주재료 이름'], str):
        row['주재료 이름'] = eval(row['주재료 이름'])
    
    if isinstance(row['주재료 이름'], list):
        for ingredient in row['주재료 이름']:
            if any(spicy in ingredient for spicy in spicy_ingredients):
                score += 3
                
    # 부재료 이름에서 맵기 성분 찾기
    if isinstance(row['부재료 이름'], str):
        row['부재료 이름'] = eval(row['부재료 이름'])
        
    if isinstance(row['부재료 이름'], list):
        for ingredient in row['부재료 이름']:
            if any(spicy in ingredient for spicy in spicy_ingredients):
                score += 2
    
    # 양념에서 맵기 성분 찾기
    if isinstance(row['양념'], str):
        row['양념'] = eval(row['양념'])
    
    if isinstance(row['양념'], list):
        for ingredient in row['양념']:
            if any(spicy in ingredient for spicy in spicy_ingredients):
                score += 1
                
    return score

data['맵기'] = data.apply(calculate_spiciness, axis=1)
data

,메뉴 이름,다이어트,주재료 이름,부재료 이름,양념,국가 분류_동남아시아,국가 분류_멕시코,국가 분류_서양,국가 분류_이탈리아,국가 분류_일본,국가 분류_중국,국가 분류_퓨전,국가 분류_한식,맵기
0,견과류 영양솥밥,0,"['백미 1과3/4컵', '은행 12개', '밤 8개', '대추 4개', '잣 1큰술']",['물 1과3/4컵'],NaN,0,0,0,0,0,0,0,1,0
1,굴밥,0,"['굴 300g', '쌀 200g', '보리쌀 80g']","['어묵(찐것) 50g', '표고버섯 3장', '당근 50g', '완두콩(통조림) ...",['소금 1작은술'],0,0,0,0,0,0,0,1,0
2,김치볶음밥,0,"['김치(잘게썬것) 1/3포기', '밥 2공기']","['감자 1/4개', '양파 1/2개', '햄 60g', '식용유 약간', '계란 ...","['소금 약간', '후춧가루 약간']",0,0,0,0,0,0,0,1,3
3,꽃상추쌈,0,"['상추 4장', '밥 1그릇']","['다진쇠고기 100g', '다진양파 1/2개']","['간장 1큰술', '설탕 1/2큰술', '다진파 약간', '다진마늘 약간', '깨...",0,0,0,0,0,0,0,1,1
4,녹두죽,1,"['녹두 1컵', '쌀 2컵']",['물 14컵'],"['소금 약간', '설탕 약간']",0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,삶은무해산물볶음,1,"['새우살 50g', '조갯살 50g', '오징어 50g', '무 1개']","['양파 1/4개', '피망 1/4개', '홍고추 1/2개']","['다시용멸치 한줌', '다시마 10 X 10cm 1장', '간장 3큰술', '굴소...",0,0,0,0,0,0,1,0,2
882,엔다이브 위의 쇠고기볶음,0,"['쇠고기 50g', '앤다이브 6잎']","['청피망 1/2개', '홍피망 1/2개', '양파 1/2개']","['올리브오일 1큰술', '굴소스 1큰술', '[양념장] 올리브오일 1큰술', '간...",0,0,0,0,0,0,1,0,0
883,피자떡볶이,1,"['햄 100g', '떡볶이떡 150g']","['어묵 2장', '피망 1/3개', '피자치즈 약간', '양파 1/3개']","['고추장 3큰술', '고춧가루 1작은술', '물엿 1큰술', '꿀 1작은술', '...",0,0,0,0,0,0,1,0,2
884,해물채소볶음,1,"['새우 30g', '바지락 50g', '파프리카 1/2개', '청경채 2개']","['조갯살 30g', '콜리플라워 30g', '가지 1/2개']","['굴소스 3큰술', '참기름 약간', '다진마늘 약간']",0,0,0,0,0,0,1,0,0


In [48]:
columns_to_drop = ['주재료 이름', '부재료 이름', '양념']
CFdata = data.drop(columns=columns_to_drop)
CFdata

,메뉴 이름,다이어트,국가 분류_동남아시아,국가 분류_멕시코,국가 분류_서양,국가 분류_이탈리아,국가 분류_일본,국가 분류_중국,국가 분류_퓨전,국가 분류_한식,맵기
0,견과류 영양솥밥,0,0,0,0,0,0,0,0,1,0
1,굴밥,0,0,0,0,0,0,0,0,1,0
2,김치볶음밥,0,0,0,0,0,0,0,0,1,3
3,꽃상추쌈,0,0,0,0,0,0,0,0,1,1
4,녹두죽,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
881,삶은무해산물볶음,1,0,0,0,0,0,0,1,0,2
882,엔다이브 위의 쇠고기볶음,0,0,0,0,0,0,0,1,0,0
883,피자떡볶이,1,0,0,0,0,0,0,1,0,2
884,해물채소볶음,1,0,0,0,0,0,0,1,0,0


In [50]:
# CSR matrix를 사용하여 데이터를 변환
csr_data = csr_matrix(CFdata.drop('메뉴 이름', axis=1).values)

# cosine similarity 계산
cosine_sim = cosine_similarity(csr_data)

# 결과를 데이터프레임으로 변환
cosine_sim_df = pd.DataFrame(cosine_sim, index=CFdata['메뉴 이름'], columns=CFdata['메뉴 이름'])

# 유사도 행렬 확인
cosine_sim_df

메뉴 이름,견과류 영양솥밥,굴밥,김치볶음밥,꽃상추쌈,녹두죽,단팥죽,달래간장과 모둠버섯밥,닭날개주먹밥,닭안심죽,대구계란죽,...,김치계란피자,김치치킨도리아,대파 파프리카굴소스볶음,돈육불고기,두부케첩볶음,삶은무해산물볶음,엔다이브 위의 쇠고기볶음,피자떡볶이,해물채소볶음,햄마늘볶음
메뉴 이름,,,,,,,,,,,,,,,,,,,,,
견과류 영양솥밥,1.000000,1.000000,0.316228,0.707107,0.707107,0.707107,0.447214,1.000000,0.707107,0.707107,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
굴밥,1.000000,1.000000,0.316228,0.707107,0.707107,0.707107,0.447214,1.000000,0.707107,0.707107,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
김치볶음밥,0.316228,0.316228,1.000000,0.894427,0.223607,0.223607,0.989949,0.316228,0.223607,0.223607,...,0.900000,0.000000,0.000000,0.000000,0.000000,0.774597,0.000000,0.774597,0.000000,0.000000
꽃상추쌈,0.707107,0.707107,0.894427,1.000000,0.500000,0.500000,0.948683,0.707107,0.500000,0.500000,...,0.670820,0.000000,0.000000,0.000000,0.000000,0.577350,0.000000,0.577350,0.000000,0.000000
녹두죽,0.707107,0.707107,0.223607,0.500000,1.000000,1.000000,0.316228,0.707107,1.000000,1.000000,...,0.000000,0.000000,0.500000,0.500000,0.500000,0.288675,0.000000,0.288675,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
삶은무해산물볶음,0.000000,0.000000,0.774597,0.577350,0.288675,0.288675,0.730297,0.000000,0.288675,0.288675,...,0.903696,0.408248,0.577350,0.577350,0.577350,1.000000,0.408248,1.000000,0.577350,0.577350
엔다이브 위의 쇠고기볶음,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.316228,1.000000,0.707107,0.707107,0.707107,0.408248,1.000000,0.408248,0.707107,0.707107
피자떡볶이,0.000000,0.000000,0.774597,0.577350,0.288675,0.288675,0.730297,0.000000,0.288675,0.288675,...,0.903696,0.408248,0.577350,0.577350,0.577350,1.000000,0.408248,1.000000,0.577350,0.577350


In [53]:
cosine_sim_df.to_csv('cosine_sim.csv', encoding='utf-8-sig')